# image prediction
project origins

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import h5py
import scipy
from PIL import Image
from scipy import ndimage




In [2]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
(X_train,Y_train), (X_test, Y_test) = datasets.cifar10.load_data()
selected_classes = [0,1,2,3,7,8]

train_filter = np.isin(Y_train, selected_classes).flatten()
X_train_filtered = X_train[train_filter]
Y_train_filtered = Y_train[train_filter]

test_filter = np.isin(Y_test, selected_classes).flatten()
X_test_filtered = X_test[test_filter]
Y_test_filtered = Y_test[test_filter]

for i, cls in enumerate(selected_classes):
    Y_train_filtered[Y_train_filtered == cls] = i  
    Y_test_filtered[Y_test_filtered == cls ] = i

X_train_filtered, X_test_filtered = X_train_filtered/255.0, X_test_filtered/255.0

print("Filtered training data shape:",X_train_filtered.shape)
print("Filtered training label shape:", Y_train_filtered.shape)
print("Filtered test data shape:", X_test_filtered.shape)
print("Filtered test label shape:", Y_test_filtered.shape)

Y_train_filtered = to_categorical(Y_train_filtered, num_classes = len(selected_classes))
Y_test_filtered = to_categorical(Y_test_filtered, num_classes = len(selected_classes))

Filtered training data shape: (30000, 32, 32, 3)
Filtered training label shape: (30000, 1)
Filtered test data shape: (6000, 32, 32, 3)
Filtered test label shape: (6000, 1)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam

def create_model():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Use softmax for multiclass classification
    model.add(Dense(len(selected_classes), activation='softmax'))
    
    return model

# Call the function to create the model
model_classy = create_model()

# Compile the model
model_classy.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Optionally, you can print the model summary
model_classy.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)                   │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 4, 4, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 307,270 (1.17 MB)

 Trainable params: 307,270 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model_classy.fit(X_train_filtered, Y_train_filtered, 
          validation_data=(X_test_filtered, Y_test_filtered), 
          epochs=10, 
          batch_size=64)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 54s 107ms/step - accuracy: 0.3226 - loss: 1.5512 - val_accuracy: 0.6097 - val_loss: 1.0058
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 81s 106ms/step - accuracy: 0.5989 - loss: 1.0340 - val_accuracy: 0.6992 - val_loss: 0.8087
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 106ms/step - accuracy: 0.6704 - loss: 0.8852 - val_accuracy: 0.7427 - val_loss: 0.7205
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 56s 119ms/step - accuracy: 0.7090 - loss: 0.7943 - val_accuracy: 0.7608 - val_loss: 0.6427
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 92s 141ms/step - accuracy: 0.7358 - loss: 0.7264 - val_accuracy: 0.7802 - val_loss: 0.6009
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 67s 143ms/step - accuracy: 0.7609 - loss: 0.6629 - val_accuracy: 0.7937 - val_loss: 0.5605
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 62s 132ms/step - accuracy: 0.7720 - loss: 0.6354 - val_accuracy: 0.8062 - val_loss: 0.5474
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 75s 117ms/step - accuracy: 0.7839 - loss: 0

In [16]:
test_loss, test_acc = model_classy.evaluate(X_test_filtered, Y_test_filtered, verbose = 2)
print("test accuracy: {test_acc}")

188/188 - 3s - 18ms/step - accuracy: 0.8165 - loss: 0.4962
test accuracy: {test_acc}


In [21]:
from tensorflow.keras.preprocessing import image  # Make sure this is imported
import numpy as np
# 1. Load your trained model
model = model_classy  # or use the correct variable name for your model

# 2. Load the image you want to test
img_path = r'C:\Users\Garba Buhari\Desktop\image prediction\車 • Instagram.jfif'
img = image.load_img(img_path, target_size=(32, 32))

# 3. Convert image to array and preprocess
img_array = image.img_to_array(img) / 255.0  # Convert to array and normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# 4. Make a prediction using your correctly named model
predictions = model_classy.predict(img_array)

# 5. Get the predicted class
predicted_class_index = np.argmax(predictions[0])
print(f"Predicted class index: {predicted_class_index}")

# Assuming you have a dictionary for class labels:
class_labels = {0: 'Class Airplane', 1: 'Class Automobile', 2: 'Class Bird', 3: 'Class Cat', 4: 'class Horse', 5: 'class Ship'}  # Update with actual labels
predicted_class_label = class_labels[predicted_class_index]
print(f'Predicted class label: {predicted_class_label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Predicted class index: 1
Predicted class label: Class Automobile
